In [122]:
import requests
import random
import re
import os
import csv
import time
from bs4 import BeautifulSoup

# 免费快代理
url = 'https://www.kuaidaili.com/free/inha/'

# 用户代理池
agent = [
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0',
        'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
        'Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
        'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
        'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
]

# 请求头部
params = {}

# csv 头部
csv_headers = ['ip','port','request_type','time']

In [119]:
class Utils(object):
    def getHeaders(self):
        headers = {
            'User-Agent' : random.sample(agent, 1)[0]
        }
        return headers
    def get_path(self):
        return os.getcwd()

In [160]:
class Spider(object):
    """
    捕获数据类
    """
    def __init__(self, url):
        self.url = url
    def send_request(self, prms = {}, url = '', page = 1):
        """
        发送请求
        :prms:请求字段
        :url:请求地址
        """
        if not url:
            url = self.url
        url = url + str(page)
        print(url)
        utils = Utils()
        request = requests.get(url, headers=utils.getHeaders(), params = prms)
        return request.text
    def parse_request(self, text):
        """
        解析文本
        :text:文本数据
        """
        soup = BeautifulSoup(text,'lxml')
        ip_list = soup.find('tbody').find_all('tr')
        data_list = []
        for data in ip_list:
            data_list.append({
                'ip' : data.find('td', attrs={"data-title": "IP"}).get_text(),
                'port' : data.find('td', attrs={"data-title": "PORT"}).get_text(),
                'request_type' : data.find('td', attrs={"data-title": "类型"}).get_text(),
                'time' : re.search(r"(\d+\.?\d*)", data.find('td',attrs={"data-title": "响应速度"}).get_text()).group(1)
            })
        return data_list

In [136]:
class Test(object):
    """
    验证类
    """
    def check_ip(self, req_type, ip, port):
        """
        测试代理ip
        :req_type:协议类型
        :ip:代理ip
        :port:端口号
        """
        utils = Utils()
        proxies = {
            req_type.lower() : req_type.lower() + '://' + ip + ':' + port
        }
        request = requests.get("https://www.baidu.com",headers = utils.getHeaders(), proxies = proxies, timeout = 1)
        return request.status_code == 200

In [163]:
class Save(object):
    def __init__(self, path):
        self.path = path
    def csv_save(self, ip_list):
        utils = Utils()
        test = Test()
        if(test.check_ip(ip_list['request_type'], ip_list['ip'], ip_list['port'])):
            with open('ip.csv',mode = 'a', newline='') as f:
                writer = csv.DictWriter(f, csv_headers)
                writer.writerow(ip_list)
        else:
            print("此ip不可用!!!")

In [165]:
# 保存可行代理ip
if __name__ == '__main__':
    spider = Spider(url)
    utils = Utils()
    save = Save(utils.get_path())
    # 存储csv头
    with open('ip.csv',mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, csv_headers)
            writer.writeheader()
    page_list = spider.parse_request(spider.send_request(params, 0))
    for ip in page_list:
        save.csv_save(ip)
        time.sleep(1)

https://www.kuaidaili.com/free/inha/1


In [162]:
# 测试re.rearch()
b = '0.2秒'
re.search(r"(\d+\.?\d*)",b).group(1)

'0.2'